The scores on the LB is a little fishy. Don't worry :)

Please **Upvote**, **Fork the LATEST VERSION**, run, submit, and enjoy!

# EDA

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
cop = ''

* There are **876** columns in `train_features.csv` file and out of these there are **875** features. 
* There are **207** labels that we need to predict for each datapoint.
* Most of features look real-valued (continuous) and few of them look are categorical type. We'll see them detail.

In [ ]:
df1 = pd.read_csv("../input/lish-moa/train_features.csv")
df2 = pd.read_csv("../input/lish-moa/train_targets_scored.csv")
df3 = pd.read_csv("../input/lish-moa/train_targets_nonscored.csv")

print("shape of train_features.csv: ", df1.shape)
print("shape of train_targets_scored.csv: ", df2.shape)
print("shape of train_targets_nonscored.csv: ", df3.shape)

print("Total number of datapoints: {:,}".format(df1.shape[0]))

df1.head()

In [ ]:
df1.describe()

## Data-type of features 

In [ ]:
# knowing the data types of each column of `train_features.csv`
data_types = df1.dtypes
unique_dtypes = data_types.unique()
print("number of dtypes in `train_features.csv`: ", len(unique_dtypes),
      "\nAnd these are: ", unique_dtypes)

Obj   = []
Int   = []
Float = []
for col, data_type in zip(df1.columns, data_types):
    if data_type == 'object':Obj.append(col)        
    elif data_type == 'int64':Int.append(col)
    elif data_type == 'float64':Float.append(col)
print("number of object data type: ", len(Obj))
print("number of int64 data type: ", len(Int))
print("number of float64 data type: ", len(Float))

assert len(Obj)+len(Int)+len(Float) == df1.shape[1]

In [ ]:
Obj, Int

* There is only one column for `int64` column and `3` for the `object` datatype.
* `cp_dose` and `cp_type` are categorical variable.
* `cp_time` has `int64` datatype, but this one is also categorical variable. 
* Let's see the number of unique values in these columns.

In [ ]:
print("Number of unique values in `cp_type` col is: {} and these are: {}"
      .format(len(df1.loc[:, "cp_type"].unique()), df1.loc[:, "cp_type"].unique()))

print("Number of unique values in `cp_dose` col is: {} and these are: {}"
      .format(len(df1.loc[:, "cp_dose"].unique()), df1.loc[:, "cp_dose"].unique()))

print("Number of unique values in `cp_time` col is: {} and these are: {}"
      .format(len(df1.loc[:, "cp_time"].unique()), df1.loc[:, "cp_time"].unique()))

* One hot encoding for `cp_type` and `cp_dose`.
* `D1` == `0`, `D2`==`1`.
* `trt_cp`== `0`, `ctl_vehicle` ==`.
* Label encoding for`cp_time`, `0` for 24, `1` for 48 and `2` for 72.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(18,6))

fig.suptitle("Count Plot of Categorical variables", fontsize = 24)
sns.countplot(x ='cp_type', data = df1, ax = axs[0])
axs[0].set_title("For: cp_type", fontsize= 16)
sns.countplot(x ='cp_dose', data = df1, ax = axs[1])
axs[1].set_title("For: cp_dose", fontsize = 16)
axs[1].set(ylabel = '')
sns.countplot(x = 'cp_time', data = df1, ax = axs[2])
axs[2].set_title("For: cp_time", fontsize = 16)
axs[2].set(ylabel = '')

plt.show()

* For both variables `cp_dose` and `cp_time` number of count is almost same for all unique values present in their columns.
* But for `cp_type` there are very data points that corresponds to `ctl_vehicle`.

**NOTE:** Use stratified spliting with `cp_type` column. This will maintain the `cp_type`'s uniques values count-ratio.  

## Encoding

In [ ]:
# cp_type, cp_time, cp_dose
def encode_cp_time(row):
    val = None
    if row == 24:val = 1
    elif row == 48:val = 2
    else:val = 3
    return val
        
df1["cp_type"] = df1["cp_type"].apply(lambda x: 0 if x=='trt_cp' else 1)
df1["cp_dose"] = df1["cp_dose"].apply(lambda x: 0 if x=='D1' else 1)
df1["cp_time"] = df1["cp_time"].apply(encode_cp_time)

df1.head()

## PCA & Correlation

* Let's normalise the features, first

In [ ]:
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.decomposition import PCA

### PCA

In [ ]:
X = df1.iloc[:, 1:]
x = StandardScaler().fit_transform(X)
x = pd.DataFrame(data=x, columns = X.columns)
x.head()

* There are `875` features, first let's reduce the dimension using **PCA**.

In [ ]:
pca = PCA(n_components=None, svd_solver = 'full')
pca.fit_transform(x)
var = pca.explained_variance_ratio_

In [ ]:
for i in range(1,9):
    print("Variance explained by top {} components: {}".format(i*100, var[:i*100].sum()))

* I would always prefer high variance explained (between 95-99%). But for that, I have to take atleast 600 components.
* 800 components are explainig the 99.41% (approx) variance.
* Now, one can select 100, 200, 300, .... any number of features and see the performance of model.

### Correlation

**Source:** https://www.geeksforgeeks.org/exploring-correlation-in-python/

In [ ]:
corrmat = X.corr()

In [ ]:
f, ax = plt.subplots(1,1, figsize =(18, 8))
sns.heatmap(corrmat, ax = ax)
plt.show()

* From above plot, it can be see that the features that starts with `c-`  looks highly correlated to each other. These are also shwoing strong correaltion with other features also.
* Let's see the heatmap of features that starts with `c-`.

In [ ]:
cols = [col for col in corrmat.columns if col.startswith('c-')]
f, ax = plt.subplots(1,1, figsize =(18, 8))
sns.heatmap(corrmat.loc[cols, cols], ax = ax)
plt.show()

* The most of thefeatures that start with `c-` are highly correlated.

**NOTE:** If you're using all features, use tree-based model or high dropout in first (just after input) in neural-network architecture.

# Random Model

* This will predict the random value between 0 and 1 (inclusive, i.e. 0<=prob<=1) for each label of each datapoints.
* We'll calculate `the worst model` performance. 

In [ ]:
from sklearn.metrics import log_loss
def total_loss(y_true, y_pred):
    """
    y_true: numpy nd-array of shape (None , 206), None means any value
    y_pred: numpy nd-array of shape (None , 206)
    """
    losses = []
    for i in range(y_true.shape[1]):losses.append(log_loss(y_true[:,i], y_pred[:,i], eps=1e-15))
    return np.mean(losses)

In [ ]:
df_test = pd.read_csv("../input/lish-moa/test_features.csv")
print("number of test datapoints: {:,}".format(df_test.shape[0]))

y_train = df2.iloc[:, 1:].values
y_train_pred = np.random.random_sample(y_train.shape) 
y_test_pred = np.random.random_sample((df_test.shape[0], y_train.shape[1])) 

In [ ]:
tr_loss = total_loss(y_train, y_train_pred)
print("train loss: ", tr_loss)

In [ ]:
test_df = pd.DataFrame(data = y_test_pred, columns = df2.columns[1:])
temp = pd.DataFrame(data=df_test.loc[:, 'sig_id'])
test_df = pd.concat([temp, test_df], ignore_index=False, axis=1)
test_df['5-alpha_reductase_inhibitor'] = test_df['5-alpha_reductase_inhibitor']  + 1.2
test_df.head()

In [ ]:
test_df.to_csv("./submission.csv",index=False)

In [ ]:
import numpy as np
import pandas as pd
df1 = pd.read_csv('../input/moa-blending/0.01832.csv')
df2 = pd.read_csv('../input/mypredpublic/my_predictions_0000001.csv')
df3 = pd.read_csv('../input/moa-blending/0.01833.csv')
def order_sub(sub) : 
    return sub.sort_values('sig_id').reset_index(drop=True)
df1 = order_sub(df1)
df2 = order_sub(df2)
df3 = order_sub(df3)
BLEND=df1.copy()
BLEND.iloc[:,1:] =  df1.iloc[:,1:]*0.6 +df2.iloc[:,1:]*0.2 +df3.iloc[:,1:]*0.2
TARGET_COL = ['5-alpha_reductase_inhibitor', '11-beta-hsd1_inhibitor', 'acat_inhibitor', 'acetylcholine_receptor_agonist', 'acetylcholine_receptor_antagonist', 'acetylcholinesterase_inhibitor', 'adenosine_receptor_agonist', 'adenosine_receptor_antagonist', 'adenylyl_cyclase_activator', 'adrenergic_receptor_agonist', 'adrenergic_receptor_antagonist', 'akt_inhibitor', 'aldehyde_dehydrogenase_inhibitor', 'alk_inhibitor', 'ampk_activator', 'analgesic', 'androgen_receptor_agonist', 'androgen_receptor_antagonist', 'anesthetic_-_local', 'angiogenesis_inhibitor', 'angiotensin_receptor_antagonist', 'anti-inflammatory', 'antiarrhythmic', 'antibiotic', 'anticonvulsant', 'antifungal', 'antihistamine', 'antimalarial', 'antioxidant', 'antiprotozoal', 'antiviral', 'apoptosis_stimulant', 'aromatase_inhibitor', 'atm_kinase_inhibitor', 'atp-sensitive_potassium_channel_antagonist', 'atp_synthase_inhibitor', 'atpase_inhibitor', 'atr_kinase_inhibitor', 'aurora_kinase_inhibitor', 'autotaxin_inhibitor', 'bacterial_30s_ribosomal_subunit_inhibitor', 'bacterial_50s_ribosomal_subunit_inhibitor', 'bacterial_antifolate', 'bacterial_cell_wall_synthesis_inhibitor', 'bacterial_dna_gyrase_inhibitor', 'bacterial_dna_inhibitor', 'bacterial_membrane_integrity_inhibitor', 'bcl_inhibitor', 'bcr-abl_inhibitor', 'benzodiazepine_receptor_agonist', 'beta_amyloid_inhibitor', 'bromodomain_inhibitor', 'btk_inhibitor', 'calcineurin_inhibitor', 'calcium_channel_blocker', 'cannabinoid_receptor_agonist', 'cannabinoid_receptor_antagonist', 'carbonic_anhydrase_inhibitor', 'casein_kinase_inhibitor', 'caspase_activator', 'catechol_o_methyltransferase_inhibitor', 'cc_chemokine_receptor_antagonist', 'cck_receptor_antagonist', 'cdk_inhibitor', 'chelating_agent', 'chk_inhibitor', 'chloride_channel_blocker', 'cholesterol_inhibitor', 'cholinergic_receptor_antagonist', 'coagulation_factor_inhibitor', 'corticosteroid_agonist', 'cyclooxygenase_inhibitor', 'cytochrome_p450_inhibitor', 'dihydrofolate_reductase_inhibitor', 'dipeptidyl_peptidase_inhibitor', 'diuretic', 'dna_alkylating_agent', 'dna_inhibitor', 'dopamine_receptor_agonist', 'dopamine_receptor_antagonist', 'egfr_inhibitor', 'elastase_inhibitor', 'erbb2_inhibitor', 'estrogen_receptor_agonist', 'estrogen_receptor_antagonist', 'faah_inhibitor', 'farnesyltransferase_inhibitor', 'fatty_acid_receptor_agonist', 'fgfr_inhibitor', 'flt3_inhibitor', 'focal_adhesion_kinase_inhibitor', 'free_radical_scavenger', 'fungal_squalene_epoxidase_inhibitor', 'gaba_receptor_agonist', 'gaba_receptor_antagonist', 'gamma_secretase_inhibitor', 'glucocorticoid_receptor_agonist', 'glutamate_inhibitor', 'glutamate_receptor_agonist', 'glutamate_receptor_antagonist', 'gonadotropin_receptor_agonist', 'gsk_inhibitor', 'hcv_inhibitor', 'hdac_inhibitor', 'histamine_receptor_agonist', 'histamine_receptor_antagonist', 'histone_lysine_demethylase_inhibitor', 'histone_lysine_methyltransferase_inhibitor', 'hiv_inhibitor', 'hmgcr_inhibitor', 'hsp_inhibitor', 'igf-1_inhibitor', 'ikk_inhibitor', 'imidazoline_receptor_agonist', 'immunosuppressant', 'insulin_secretagogue', 'insulin_sensitizer', 'integrin_inhibitor', 'jak_inhibitor', 'kit_inhibitor', 'laxative', 'leukotriene_inhibitor', 'leukotriene_receptor_antagonist', 'lipase_inhibitor', 'lipoxygenase_inhibitor', 'lxr_agonist', 'mdm_inhibitor', 'mek_inhibitor', 'membrane_integrity_inhibitor', 'mineralocorticoid_receptor_antagonist', 'monoacylglycerol_lipase_inhibitor', 'monoamine_oxidase_inhibitor', 'monopolar_spindle_1_kinase_inhibitor', 'mtor_inhibitor', 'mucolytic_agent', 'neuropeptide_receptor_antagonist', 'nfkb_inhibitor', 'nicotinic_receptor_agonist', 'nitric_oxide_donor', 'nitric_oxide_production_inhibitor', 'nitric_oxide_synthase_inhibitor', 'norepinephrine_reuptake_inhibitor', 'nrf2_activator', 'opioid_receptor_agonist', 'opioid_receptor_antagonist', 'orexin_receptor_antagonist', 'p38_mapk_inhibitor', 'p-glycoprotein_inhibitor', 'parp_inhibitor', 'pdgfr_inhibitor', 'pdk_inhibitor', 'phosphodiesterase_inhibitor', 'phospholipase_inhibitor', 'pi3k_inhibitor', 'pkc_inhibitor', 'potassium_channel_activator', 'potassium_channel_antagonist', 'ppar_receptor_agonist', 'ppar_receptor_antagonist', 'progesterone_receptor_agonist', 'progesterone_receptor_antagonist', 'prostaglandin_inhibitor', 'prostanoid_receptor_antagonist', 'proteasome_inhibitor', 'protein_kinase_inhibitor', 'protein_phosphatase_inhibitor', 'protein_synthesis_inhibitor', 'protein_tyrosine_kinase_inhibitor', 'radiopaque_medium', 'raf_inhibitor', 'ras_gtpase_inhibitor', 'retinoid_receptor_agonist', 'retinoid_receptor_antagonist', 'rho_associated_kinase_inhibitor', 'ribonucleoside_reductase_inhibitor', 'rna_polymerase_inhibitor', 'serotonin_receptor_agonist', 'serotonin_receptor_antagonist', 'serotonin_reuptake_inhibitor', 'sigma_receptor_agonist', 'sigma_receptor_antagonist', 'smoothened_receptor_antagonist', 'sodium_channel_inhibitor', 'sphingosine_receptor_agonist', 'src_inhibitor', 'steroid', 'syk_inhibitor', 'tachykinin_antagonist', 'tgf-beta_receptor_inhibitor', 'thrombin_inhibitor', 'thymidylate_synthase_inhibitor', 'tlr_agonist', 'tlr_antagonist', 'tnf_inhibitor', 'topoisomerase_inhibitor', 'transient_receptor_potential_channel_antagonist', 'tropomyosin_receptor_kinase_inhibitor', 'trpv_agonist', 'trpv_antagonist', 'tubulin_inhibitor', 'tyrosine_kinase_inhibitor', 'ubiquitin_specific_protease_inhibitor', 'vegfr_inhibitor', 'vitamin_b', 'vitamin_d_receptor_agonist', 'wnt_inhibitor']
NUM_TARGET = len(TARGET_COL)
submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')
df = pd.read_csv("/kaggle/input/lish-moa/sample_submission.csv")

public_id = list(df['sig_id'].values)

df_test = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
test_id = list(df_test['sig_id'].values)

private_id = list(set(test_id)-set(public_id))

df_submit = pd.DataFrame(index = public_id+private_id, columns=TARGET_COL)
df_submit.index.name = 'sig_id'
df_submit[:] = 0
df_predict = BLEND.copy()
df_submit.loc[df_predict.sig_id,:] = df_predict[TARGET_COL].values
df_submit.loc[df_test[df_test.cp_type=='ctl_vehicle'].sig_id]= 0
df_submit.to_csv(cop+ 'submission.csv',index=True)

**Splitting the data**

```Python
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack

from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
df_train = df1.merge(df2, on = 'sig_id')
print("shape: ", df_train.shape)
df_train.head()

train, val = train_test_split(df_train, test_size = 0.2, random_state=42,
                             stratify = df_train["cp_type"])
X_train, Y_train = df_train.iloc[:, :-206], df_train.iloc[:, -206:]
X_val, Y_val = val.iloc[:, :-206], val.iloc[:, -206:]

print("Number of datapoints in train-set: {:,}".format(len(X_train)))
print("Number of datapoints in val-set: {:,}".format(len(X_val)))
```